In [13]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import numpy as np
import pandas as pd

# import data types
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# set up the session
spark = SparkSession.builder \
        .master("local") \
        .appName("diabetes_classifier") \
        .getOrCreate()

In [14]:
#load all data files
#pandas is needed to read .xpt files then spark is used to create a dataframe

demo = spark.createDataFrame(pd.read_sas("DEMO_J.XPT") \
                             .drop(columns = ['SDDSRVYR', 'RIDSTATR', 'RIDAGEMN',
                                           'RIDRETH1', 'RIDEXMON', 'RIDEXAGM', 'DMQMILIZ', 'DMQADFC',
                                           'DMDBORN4', 'DMDCITZN', 'DMDYRSUS', 'DMDEDUC3', 'DMDMARTL',
                                           'RIDEXPRG', 'SIALANG', 'SIAPROXY', 'SIAINTRP', 'FIALANG', 'FIAPROXY',
                                           'FIAINTRP', 'MIALANG', 'MIAPROXY', 'MIAINTRP', 'AIALANGA', 'DMDHHSIZ',
                                           'DMDFMSIZ', 'DMDHHSZA', 'DMDHHSZB', 'DMDHHSZE', 'DMDHRGND', 'DMDHRAGZ',
                                           'DMDHREDZ', 'DMDHRMAZ', 'DMDHSEDZ', 'WTINT2YR', 'WTMEC2YR', 'SDMVPSU',
                                           'SDMVSTRA', 'INDFMIN2', 'INDFMPIR']) \
                             .rename({'SEQN': 'ParticipantID', 
                                    'RIAGENDR': 'Gender',
                                    'RIDAGEYR': 'Age',
                                    'RIDRETH3': 'Race',
                                    'DMDEDUC2': 'Education_Level',
                                    'INDHHIN2': 'Household_income'}, axis = 1))

diet_1 = spark.createDataFrame(pd.read_sas("DR1IFF_J.XPT")\
                              .drop(columns = ['WTDRD1', 'WTDR2D', 'DR1ILINE', 'DR1DRSTZ', 'DR1EXMER', 'DRABF',
                                               'DRDINT', 'DR1DBIH', 'DR1DAY', 'DR1LANG', 'DR1CCMNM', 'DR1CCMTX',
                                               'DR1_020', 'DR1_030Z', 'DR1FS', 'DR1_040Z', 'DR1IFDCD', 'DR1IGRMS',
                                               'DR1IATOC', 'DR1IATOA',
                                               'DR1IRET', 'DR1IVARA', 'DR1IACAR', 'DR1IBCAR', 'DR1ICRYP', 'DR1ILYCO',
                                               'DR1ILZ', 'DR1IVB1', 'DR1IVB2', 'DR1INIAC', 'DR1IVB6', 'DR1IFOLA',
                                               'DR1IFA', 'DR1IFF', 'DR1IFDFE', 'DR1ICHL', 'DR1IVB12', 'DR1IB12A',
                                               'DR1IVC', 'DR1IVD', 'DR1IVK', 'DR1ICALC', 'DR1IPHOS', 'DR1IMAGN',
                                               'DR1IIRON', 'DR1IZINC', 'DR1ICOPP', 'DR1ISODI', 'DR1IPOTA', 'DR1ISELE',
                                               'DR1ICAFF', 'DR1ITHEO', 'DR1IMOIS', 'DR1IS040', 'DR1IS060',
                                               'DR1IS080', 'DR1IS100', 'DR1IS120', 'DR1IS140', 'DR1IS160', 'DR1IS180',
                                               'DR1IM161', 'DR1IM181', 'DR1IM201', 'DR1IM221', 'DR1IP182', 'DR1IP183',
                                               'DR1IP184', 'DR1IP204', 'DR1IP205', 'DR1IP225', 'DR1IP226']) \
                               .rename({'SEQN': 'ParticipantID', 
                                        'DR1IKCAL': 'Energy',
                                        'DR1IPROT': 'Protein',
                                        'DR1ICARB': 'Carbohydrates',
                                        'DR1ISUGR': 'Total_sugar',
                                        'DR1IFIBE': 'Fiber',
                                        'DR1ITFAT': 'Total_fat',
                                        'DR1ISFAT': 'Sat_fat',
                                        'DR1IMFAT': 'Monounsat_fat',
                                        'DR1IPFAT': 'Polyunsat_fat',
                                        'DR1ICHOL': 'cholesterol',
                                        'DR1IALCO': 'Alcohol'}, axis = 1))

bp = spark.createDataFrame(pd.read_sas("BPX_J.XPT") \
                          .drop(columns = ['PEASCCT1', 'BPXCHR', 'BPAARM', 'BPACSZ', 'BPXPLS',
                                           'BPXPTY', 'BPXML1', 'BPXSY1', 'BPXDI1', 'BPAEN1', 
                                           'BPAEN2', 'BPXSY3', 'BPXDI3', 'BPAEN3', 'BPXSY4', 
                                           'BPXDI4', 'BPAEN4']) \
                          .rename({'SEQN' : 'ParticipantID', 
                                    'BPXPULS' : 'Pulse', #regular = 1, irregular = 2
                                    'BPXSY2' : 'SysBP',
                                    'BPXDI2' : 'DiasBP'}, axis = 1))

bm = spark.createDataFrame(pd.read_sas("BMX_J.XPT") \
                          .drop(columns = ['BMDSTATS', 'BMIWT', 'BMXRECUM', 'BMIRECUM', 'BMXHEAD',
                                           'BMIHEAD', 'BMXHT', 'BMIHT', 'BMXLEG', 'BMILEG', 'BMXARML',
                                           'BMIARML', 'BMXARMC', 'BMIARMC', 'BMIWAIST', 'BMXHIP','BMIHIP']) \
                          .rename({'SEQN': 'ParticipantID',
                                    'BMXWT': 'Weight(kg)',
                                    'BMXBMI': 'BMI',
                                    'BMXWAIST': 'Waist_Circum'}, axis = 1))

ins = spark.createDataFrame(pd.read_sas("INS_J.XPT") \
                           .drop(columns = ['WTSAF2YR', 'LBDINSI', 'LBDINLC']) \
                           .rename({'SEQN': 'ParticipantID',
                                    'LBXIN': 'Insulin'}, axis = 1))

glu = spark.createDataFrame(pd.read_sas("GLU_J.XPT") \
                           .drop(columns = ['WTSAF2YR', 'LBXGLU']) \
                           .rename({'SEQN': 'ParticipantID',
                                  'LBDGLUSI':'Glucose'}, axis = 1))

alc = spark.createDataFrame(pd.read_sas("ALQ_J.XPT") \
                           .drop(columns = ['ALQ111', 'ALQ121', 'ALQ142', 'ALQ170']) \
                           .rename({'SEQN': 'ParticipantID',
                                  'ALQ130':'Avg_Drinks',
                                  'ALQ270':'4-5_Drinks',
                                  'ALQ280':'8+Drinks',
                                  'ALQ290':'12+Drinks',
                                  'ALQ151':'4-5DrinksDaily'}, axis = 1))

bpq = spark.createDataFrame(pd.read_sas("BPQ_J.XPT") \
                           .drop(columns = ['BPQ030', 'BPD035', 'BPQ040A', 'BPQ050A',
                                           'BPQ060', 'BPQ070', 'BPQ090D', 'BPQ100D']) \
                           .rename({'SEQN': 'ParticipantID',
                                  'BPQ020': 'HighBP', #1 = Yes, 2 = no
                                  'BPQ080': 'HighChol'},axis = 1)) #1 = Yes, 2 = No
    
pa = spark.createDataFrame(pd.read_sas("PAQ_J.XPT") \
                           .drop(columns = ['PAQ610', 'PAD615', 'PAQ625', 'PAD630',
                                           'PAQ640', 'PAD645', 'PAQ655', 'PAD660',
                                           'PAQ670', 'PAD675', 'PAD680'])\
                          .rename({'SEQN': 'ParticipantID',
                                    'PAQ605': 'VigWork',
                                    'PAQ620': 'ModWork',
                                    'PAQ635': 'Walk_bike',
                                    'PAQ650': 'VigActivity',
                                    'PAQ665': 'ModActivity'},axis = 1))
    
diab = spark.createDataFrame(pd.read_sas("DIQ_J.XPT") \
                             .drop(columns = ['DIQ172', 'DIQ175C', 'DIQ175D', 'DIQ175E', 'DIQ175F', 'DIQ175G',
                                               'DIQ175H', 'DIQ175I', 'DIQ175J', 'DIQ175K', 'DIQ175L', 'DIQ175M',
                                               'DIQ175N', 'DIQ175O', 'DIQ175P', 'DIQ175Q', 'DIQ175R', 'DIQ175S',
                                               'DIQ175T', 'DIQ175U', 'DIQ175V', 'DIQ175W', 'DIQ175X',
                                               'DIQ050', 'DID060', 'DIQ060U', 'DIQ070', 'DIQ230', 'DIQ240', 'DID250',
                                               'DID260', 'DIQ260U', 'DIQ275', 'DIQ280', 'DIQ291', 'DIQ300S', 'DIQ300D',
                                               'DID310S', 'DID310D', 'DID320', 'DID330', 'DID341', 'DID350', 'DIQ350U',
                                               'DIQ360', 'DIQ080', 'DIQ175B']) \
                            .rename({'SEQN': 'ParticipantID',
                                    'DIQ010': 'Diagnosis', #label 
                                    'DID040': 'Diagnosed_age',
                                    'DIQ160': 'Prediabetes',
                                    'DIQ170': 'Diabetes_risk',
                                    'DIQ175A': 'Fam_hist',
                                    'DIQ180': 'Blood_test'}, #1 = yes, 2 = no
                                    axis = 1))

smq = spark.createDataFrame(pd.read_sas("SMQ_J.XPT") \
                            .drop(columns = ['SMD030', 'SMQ050Q', 'SMQ050U', 'SMD057',
                                           'SMQ078', 'SMD641', 'SMD650', 'SMD093', 'SMDUPCA', 'SMD100BR',
                                           'SMD100FL', 'SMD100MN', 'SMD100LN', 'SMD100TR', 'SMD100NI', 'SMD100CO',
                                           'SMQ621', 'SMD630', 'SMQ661', 'SMQ665A', 'SMQ665B', 'SMQ665C',
                                           'SMQ665D', 'SMQ670', 'SMQ848', 'SMQ852Q', 'SMQ852U', 'SMQ895',
                                           'SMQ905', 'SMQ915', 'SMAQUEX2']) \
                           .rename({'SEQN': 'ParticipantID',
                                  'SMQ020': '100Cigs',
                                  'SMQ040': 'Smoke_Cigs', #1 = Yes, 2 = No
                                  'SMQ890': 'Smoke_Cigar',
                                  'SMQ900': 'E_cig',
                                  'SMQ910': 'Smokeless_tobacco'},axis = 1))

In [15]:
#join all dataframes on participant ID
df = demo.join(bp, on=['ParticipantID'], how='left').join(diet_1, on=['ParticipantID'], how='left').join(bm, on=['ParticipantID'], how='left').join(ins, on=['ParticipantID'], how='left') \
.join(glu, on=['ParticipantID'], how='left').join(alc, on=['ParticipantID'], how='left').join(bpq, on=['ParticipantID'], how='left').join(pa, on=['ParticipantID'], how='left') \
.join(smq, on=['ParticipantID'], how='left').join(diab, on=['ParticipantID'], how='left')

In [16]:
# examine schema
df.printSchema()

root
 |-- ParticipantID: double (nullable = true)
 |-- Gender: double (nullable = true)
 |-- Age: double (nullable = true)
 |-- Race: double (nullable = true)
 |-- Education_Level: double (nullable = true)
 |-- Household_income: double (nullable = true)
 |-- Pulse: double (nullable = true)
 |-- SysBP: double (nullable = true)
 |-- DiasBP: double (nullable = true)
 |-- Energy: double (nullable = true)
 |-- Protein: double (nullable = true)
 |-- Carbohydrates: double (nullable = true)
 |-- Total_sugar: double (nullable = true)
 |-- Fiber: double (nullable = true)
 |-- Total_fat: double (nullable = true)
 |-- Sat_fat: double (nullable = true)
 |-- Monounsat_fat: double (nullable = true)
 |-- Polyunsat_fat: double (nullable = true)
 |-- cholesterol: double (nullable = true)
 |-- Alcohol: double (nullable = true)
 |-- Weight(kg): double (nullable = true)
 |-- BMI: double (nullable = true)
 |-- Waist_Circum: double (nullable = true)
 |-- Insulin: double (nullable = true)
 |-- Glucose: double

In [24]:
# we need to fix a lot of these to be ints or floats
df.dtypes

[('ParticipantID', 'double'),
 ('Gender', 'double'),
 ('Age', 'double'),
 ('Race', 'double'),
 ('Education_Level', 'double'),
 ('Household_income', 'double'),
 ('Pulse', 'double'),
 ('SysBP', 'double'),
 ('DiasBP', 'double'),
 ('Energy', 'double'),
 ('Protein', 'double'),
 ('Carbohydrates', 'double'),
 ('Total_sugar', 'double'),
 ('Fiber', 'double'),
 ('Total_fat', 'double'),
 ('Sat_fat', 'double'),
 ('Monounsat_fat', 'double'),
 ('Polyunsat_fat', 'double'),
 ('cholesterol', 'double'),
 ('Alcohol', 'double'),
 ('Weight(kg)', 'double'),
 ('BMI', 'double'),
 ('Waist_Circum', 'double'),
 ('Insulin', 'double'),
 ('Glucose', 'double'),
 ('Avg_Drinks', 'double'),
 ('4-5_Drinks', 'double'),
 ('8+Drinks', 'double'),
 ('12+Drinks', 'double'),
 ('4-5DrinksDaily', 'double'),
 ('HighBP', 'double'),
 ('HighChol', 'double'),
 ('VigWork', 'double'),
 ('ModWork', 'double'),
 ('Walk_bike', 'double'),
 ('VigActivity', 'double'),
 ('ModActivity', 'double'),
 ('100Cigs', 'double'),
 ('Smoke_Cigs', 'double

In [41]:
df = df.drop().dropDuplicates()
df

AttributeError: 'DataFrame' object has no attribute 'isnull'

In [39]:
# all columns we want
cols = ['Diagnosis',
        'ParticipantID',
       'Gender',
       'Age',
       'Race',
       'Fam_hist',
       'Smoke_Cigs',
       'Glucose',
       'BMI',
       'SysBP',
       'DiasBP',
       'Avg_Drinks']
data = df.select(cols)


# set gender to 0/1 instead of 1/2
data = data.withColumn('Gender', data.Gender - 1)

# make diagnosis int
data = data.withColumn('Diagnosis', data.Diagnosis.cast('int'))

# the columns we need to one hot encode
cats = ['Fam_hist',
       'Smoke_Cigs',
       'Race']

# one hot encode
# will streamline much of this into pipeline later
for col in cats:
    #indexer = StringIndexer(inputCol=col,
                           #outputCol=col + '_id')
    #model = indexer.fit(data)
    #indexed = model.transform(data)
    
    encoder = OneHotEncoder(inputCol=col,
                           outputCol=col+'_vec')
    data = encoder.transform(data)

# final columns in our df that we need
cols = ['ParticipantID',
        'Diagnosis',
       'Gender',
       'Age',
       'Race_vec',
       'Fam_hist_vec',
       'Smoke_Cigs_vec',
       'Glucose',
       'BMI',
       'SysBP',
       'DiasBP',
       'Avg_Drinks']
data = data.select(cols)

Py4JJavaError: An error occurred while calling o2914.transform.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 405.0 failed 1 times, most recent failure: Lost task 7.0 in stage 405.0 (TID 12312, localhost, executor driver): java.lang.AssertionError: assertion failed: OneHotEncoder only supports up to 2147483647 indices, but got NaN.
	at scala.Predef$.assert(Predef.scala:170)
	at org.apache.spark.ml.feature.OneHotEncoderCommon$$anonfun$12$$anonfun$apply$2.apply$mcVI$sp(OneHotEncoderEstimator.scala:491)
	at scala.collection.immutable.Range.foreach$mVc$sp(Range.scala:160)
	at org.apache.spark.ml.feature.OneHotEncoderCommon$$anonfun$12.apply(OneHotEncoderEstimator.scala:489)
	at org.apache.spark.ml.feature.OneHotEncoderCommon$$anonfun$12.apply(OneHotEncoderEstimator.scala:488)
	at scala.collection.TraversableOnce$$anonfun$foldLeft$1.apply(TraversableOnce.scala:157)
	at scala.collection.TraversableOnce$$anonfun$foldLeft$1.apply(TraversableOnce.scala:157

In [36]:
data.take(1)

Py4JJavaError: An error occurred while calling o2745.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 393.0 failed 1 times, most recent failure: Lost task 0.0 in stage 393.0 (TID 12183, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$9: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage31.agg_doAggregateWithKeysOutput_1$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage31.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:255)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(Spa

In [19]:
# remove duplicates, nulls, and id
data = data.drop().dropDuplicates().select(cols[1:])

In [20]:
# assemble into feature vectors
assembler = VectorAssembler(inputCols=cols[2:], outputCol='features')
assembled = assembler.setHandleInvalid('skip') \
    .transform(data) \
    .select(['Diagnosis','features'])

In [21]:
# test-train split
(trainingData, testData) = assembled.randomSplit([0.7, 0.3])

In [22]:
# build random forest classifier
rf = RandomForestClassifier(labelCol='Diagnosis', featuresCol='features', numTrees=10)
model = rf.fit(trainingData)

Py4JJavaError: An error occurred while calling o1330.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 125.0 failed 1 times, most recent failure: Lost task 4.0 in stage 125.0 (TID 3628, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$9: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage31.agg_doAggregateWithKeys_1$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage31.agg_doAggregateWithKeys_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage31.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage31.agg_doAggregateWithoutKey_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenS

In [ ]:
# make predictions
preds = model.transform(testData)

In [ ]:
preds.show(5)

In [ ]:
# evaluate for accuracy
preds.select(['Diagnosis','prediction']) \
    .rdd \
    .map(lambda x: x[0] == x[1]) \
    .sum() / preds.count()